In [1]:
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
from animal_shelter import AnimalShelter
from PIL import Image, ImageDraw, ImageFont
import io
import base64

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "Ray247"
host = "nv-desktop-services.apporto.com"
port = 34893
database_name = "AAC"
collection_name = "animals"

# Connect to MongoDB using AnimalShelter class
crud = AnimalShelter(username, password, host, port, database_name, collection_name)

# Retrieve all data from MongoDB
df = pd.DataFrame()
if crud.collection is not None:
    mongo_data = crud.read({})
    try:
        df = pd.DataFrame.from_records(mongo_data)
        if '_id' in df.columns:
            df.drop(columns=['_id'], inplace=True)
    except Exception as e:
        print(f"Error creating DataFrame: {e}")
        df = pd.DataFrame()
else:
    print("Error: Could not establish MongoDB connection.")

#########################
# Logo Creation
#########################
def create_animal_logo(text="AAC Animals"):
    width = 300
    height = 100
    bg_color = (200, 220, 255)  # Light blue background
    text_color = (50, 50, 50)    # Dark grey text

    img = Image.new('RGB', (width, height), bg_color)
    d = ImageDraw.Draw(img)

    try:
        # Attempt to load a TrueType font with a larger size
        font = ImageFont.truetype("DejaVuSans.ttf", 24)

    except IOError:
        # Use the default font if TrueType font fails
        font = ImageFont.load_default()

    # Calculate text size and position
    text_width, text_height = d.textsize(text, font=font)
    text_x = (width - text_width) // 2
    text_y = (height - text_height) // 2

    # Draw text
    d.text((text_x, text_y), text, fill=text_color, font=font)

    # Draw simple paw print shapes further toward the edges
    paw_color = (100, 100, 100)
    paw_size = 15
    paw_centers = [(35, 60), (265, 60)]  # Moved centers closer to the edges

    for center_x, center_y in paw_centers:
        # Palm
        d.ellipse((center_x - paw_size * 0.7, center_y - paw_size, center_x + paw_size * 0.7, center_y + paw_size), fill=paw_color)
        # Fingers on one side
        finger_offset = paw_size * 0.6
        finger_spacing = paw_size * 0.3
        d.ellipse((center_x - finger_offset - finger_spacing, center_y - paw_size * 1.3, center_x - finger_offset + finger_spacing * 0, center_y - paw_size * 0.8), fill=paw_color)
        d.ellipse((center_x - finger_offset + finger_spacing * 1, center_y - paw_size * 1.3, center_x - finger_offset + finger_spacing * 2, center_y - paw_size * 0.8), fill=paw_color)
        d.ellipse((center_x - finger_offset + finger_spacing * 3, center_y - paw_size * 1.3, center_x - finger_offset + finger_spacing * 4, center_y - paw_size * 0.8), fill=paw_color)
        # Thumb on the other side
        d.ellipse((center_x + finger_offset, center_y - paw_size * 1.2, center_x + finger_offset + paw_size * 0.5, center_y - paw_size * 0.7), fill=paw_color)


    # Convert image to base64
    img_byte_arr = io.BytesIO()
    img.save(img_byte_arr, format='PNG')
    encoded_image = base64.b64encode(img_byte_arr.getvalue()).decode('ascii')
    return f'data:image/png;base64,{encoded_image}'

logo_url = create_animal_logo()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('AnimalCenterDashboard')

app.layout = html.Div([
    html.Center([
        html.Img(src=logo_url, height='80px'),
        html.B(html.H1('Austin Animal Center Outcomes'))
    ]),
    html.Hr(),
    html.Div([
        html.Label("Age Upon Outcome"),
dcc.Dropdown(
    id='age-dropdown',
    options=[
        {'label': age, 'value': age} for age in sorted(df['age_upon_outcome'].unique(), key=lambda x: int(x.split()[0]) if x.split()[0].isdigit() else float('inf'))
    ],
    placeholder="Select Age Upon Outcome"
),

        html.Label("Breed"),
        dcc.Dropdown(
            id='breed-dropdown',
            options=[{'label': breed, 'value': breed} for breed in df['breed'].unique()],
            placeholder="Select Breed"
        ),
        html.Label("Animal Type"),
        dcc.Dropdown(
            id='type-dropdown',
            options=[{'label': animal_type, 'value': animal_type} for animal_type in df['animal_type'].unique()],
            placeholder="Select Animal Type"
        ),
        html.Br(),
        html.Div(id='filtered-results'),
        html.Label("Visual Representation of Animal Types"),
        dcc.Graph(id='animal-type-graph')  # Graph for animal type counts
    ]),
    html.Hr(),
    html.P("Input specific search options in the first row of each category.(First letters of each word in animal names, types and breeds must be capitolized as seen below)", style={'fontSize': '15px', 'color': 'blue'}),
dash_table.DataTable(
    id='datatable-id',
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
    data=df.to_dict('records'),
    page_size=10,
    sort_action='native',  # Enable sorting
    filter_action='native',  # Enable filtering
    page_action='native',  # Enable pagination
    fixed_rows={'headers': True},
    style_table={'height': '400px', 'overflowY': 'auto'},
    style_cell={'textAlign': 'left', 'minWidth': '150px', 'width': 'auto'},
    style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
),

    html.Br(),
    html.Div(id='map-id', style={'height': '500px'})  # Map container
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('filtered-results', 'children'),
     Output('animal-type-graph', 'figure')],
    [Input('age-dropdown', 'value'),
     Input('breed-dropdown', 'value'),
     Input('type-dropdown', 'value')]
)
def filter_data(selected_age, selected_breed, selected_type):
    filtered_df = df.copy()
    if selected_age:
        filtered_df = filtered_df[filtered_df['age_upon_outcome'] == selected_age]
    if selected_breed:
        filtered_df = filtered_df[filtered_df['breed'] == selected_breed]
    if selected_type:
        filtered_df = filtered_df[filtered_df['animal_type'] == selected_type]

    # Create the graph for animal type counts
    animal_type_counts = filtered_df['animal_type'].value_counts()
    figure = {
        'data': [
            {
                'x': animal_type_counts.index,
                'y': animal_type_counts.values,
                'type': 'bar',
                'name': 'Animal Type Counts'
            }
        ],
        'layout': {
            'title': 'Number of Each Animal Type',
            'xaxis': {'title': 'Animal Type'},
            'yaxis': {'title': 'Count'}
        }
    }
    
    return html.P(f"Filtered {len(filtered_df)} records."), figure

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "data"),
     Input('datatable-id', "active_cell")]
)
def update_map(data, active_cell):
    if active_cell and 'row' in active_cell and 'column_id' in active_cell:
        row = active_cell['row']
        selected_row = data[row]
        lat, lon = selected_row.get('location_lat'), selected_row.get('location_long')
        if lat and lon:
            geo_tile_url = "https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png"
            return dl.Map(center=[lat, lon], zoom=10, children=[
                dl.TileLayer(url=geo_tile_url, attribution="&copy; OpenTopoMap contributors"),
                dl.Marker(position=[lat, lon], draggable=False)
            ])
        else:
            return html.P("Location data is not available for this entry.")
    return html.P("Click on an animal entry to display its location on the map.")

if __name__ == '__main__':
    app.run_server(debug=True)

Connected successfully to MongoDB
Dash app running on http://127.0.0.1:29052/
